In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
import sys

sys.path.insert(0, "../")
sys.path.insert(0, "../db_config/")
sys.path.insert(0, "../helper_functions//")

from helper_functions.utilities import pandas_join_key_dual
from helper_functions.munge import (read_sdb_opleidingen, 
                                    prepare_duo_ho_files, 
                                    concat_unstack_duo_ho_files)

pd.set_option("display.max.columns", 100)
PATH_TO_RAW_DATA = '../../../data/raw/'

In [2]:
sdb = read_sdb_opleidingen(path=PATH_TO_RAW_DATA, file='studiekeuze123_all_20200417.xlsx')

INFO:root:dropping columns: 
 ['IndicatieIntensiefProgramma', 'SelectieEisenIntensiefProgramma', 'VerhoogdCollegegeldIntensiefProgramma', 'BezoekadresStraat', 'BezoekadresNummer', 'BezoekadresPostcode', 'Bekostiging', 'AssociateDegree', 'JointDegree', 'BSAopmerking', 'Studielast', 'AlgemeenOordeel', 'SourceURL', 'TitelHodex', 'Afstandonderwijs', 'Avondonderwijs', 'HeeftBindendStudieAdvies', 'HeeftExtraInstroommoment', 'ExtraStudiekosten', 'LangeBeschrijvingOpleiding', 'Opleidingswebsite', 'PercentageStudentenNaarBuitenland', 'PercentageStudentenUitBuitenland', 'PercentageDocentenUitBuitenland', 'PercentageDocentenNaarBuitenland', 'PercentageAfstandsonderwijs', 'PercentageAvondonderwijs', 'WerkgroepGrootte', 'IsKoepelOpleiding', 'StudiekeuzecheckNaam', 'StudiekeuzecheckBeleidVoor1Mei', 'StudiekeuzecheckBeleidNa1Mei', 'StudiekeuzecheckVerplicht', 'StudiekeuzecheckUrl', 'StudiekeuzecheckOmschrijving', 'HeeftHonoursProgramma', 'InstroomVoltijd', 'InstroomDeeltijd', 'InstroomDuaal', 'Instro

In [3]:
from config import DUO_HBO_CSV, DUO_WO_CSV

def prepare_duo_ho_files(path, file):
    """
    Reads in and fprmats DUO files
    """
    df = pd.read_csv(path + file, sep=";")
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(" ", "_")
    for c in ['gemeentenummer', 'opleidingscode_actueel']:
        df[c] = df[c].map("{:.0f}".format).astype(str)
    
    df['type_hoger_onderwijs'] = df['type_hoger_onderwijs'].fillna('master')
    
    rename_column_dict = {'gemeentenummer': 'gemeentenummer_duo',
                          'brin_nummer_actueel': 'brinnummer_duo',
                          'instellingsnaam_actueel': 'instellingsnaam_duo',
                          'opleidingscode_actueel': 'opleidingscode_duo',
                          'opleidingsnaam_actueel': 'opleidingsnaam_duo',
                          'type_hoger_onderwijs': 'soortopleiding_duo',
                          'opleidingsvorm': 'opleidingsvorm_duo'}
    df = df.rename(columns = rename_column_dict)
    
    df.brinnummer_duo = df.brinnummer_duo.astype(str)
    
    regex_pattern = '\s(.*)'
    df['opleidingsnaam_duo'] = df['opleidingsnaam_duo'].str.extract(regex_pattern)
    
    logging.info(f"{file} shape: {df.shape}")
    return df


def concat_unstack_duo_ho_files():
    
    hbo = prepare_duo_ho_files(path = PATH_TO_RAW_DATA, file = DUO_HBO_CSV)
    wo = prepare_duo_ho_files(path = PATH_TO_RAW_DATA, file = DUO_WO_CSV)
    duo = pd.concat([wo, hbo], axis=0)
    logging.info(f"Concatting {DUO_HBO_CSV, DUO_WO_CSV}\n. Duo file shape: {duo.shape}")
        
    gemeentes = duo.groupby("brinnummer_duo")["gemeentenummer_duo"].unique().apply(list).reset_index()
    
    logging.info('summing MAN, VROUW into TOTAL')
    duo_ = (duo.groupby(['brinnummer_duo' ,'instellingsnaam_duo', 'opleidingsnaam_duo', 
                         'croho_onderdeel', 'croho_subonderdeel', 
                        # 'opleidingsvorm_duo', 
                        # 'gemeentenummer_duo', 
                         'soortopleiding_duo','geslacht'])['2019'].sum()
            .unstack('geslacht')).reset_index(drop=False)
    duo_['tot_2019_duo'] = duo_['man'].add(duo_['vrouw'])
    
    logging.info('Merge back the concatted gemeentes')
    duo_ = pd.merge(left=duo_, right=gemeentes, on=['brinnummer_duo'], how='left')
    
    logging.info(f"Duo file shape: {duo_.shape}")
    logging.info(f'Extract \n: {duo_.head(5)}')
    
    return duo_

In [4]:
duo = concat_unstack_duo_ho_files()

INFO:root:hbo_inscriptions_2019.csv shape: (3822, 18)
INFO:root:wo_inscriptions_2019.csv shape: (3231, 18)
INFO:root:Concatting ('hbo_inscriptions_2019.csv', 'wo_inscriptions_2019.csv')
. Duo file shape: (7053, 18)
INFO:root:summing MAN, VROUW into TOTAL
INFO:root:Merge back the concatted gemeentes
INFO:root:Duo file shape: (2496, 10)
INFO:root:Extract 
:   brinnummer_duo                     instellingsnaam_duo  \
0           00DV  Protestantse Theologische Universiteit   
1           00DV  Protestantse Theologische Universiteit   
2           00DV  Protestantse Theologische Universiteit   
3           00DV  Protestantse Theologische Universiteit   
4           00IC                  Katholieke PABO Zwolle   

                    opleidingsnaam_duo  croho_onderdeel  \
0                    Gemeentepredikant  taal en cultuur   
1                       Predikantschap  taal en cultuur   
2             Theologie (joint degree)  taal en cultuur   
3                           Verdieping  taal 

In [5]:
sdb_join_cols = ['brinnummer_sdb', 'naamopleiding_sdb', 'soortopleiding_sdb']
sdb_add_cols = ['cluster_sdb', 'sector_sdb', 
                'kortebeschrijvingopleiding_sdb',
                'eerstejaarsaantal_sdb', 'studentenaantal_sdb']

In [7]:
test = pandas_join_key_dual(left_df=duo, right_df=sdb[sdb_join_cols + sdb_add_cols], 
                     left_key=['brinnummer_duo', 'opleidingsnaam_duo', 'soortopleiding_duo'], 
                     right_key=sdb_join_cols, how='inner')
test = test.drop(sdb_join_cols, axis=1)

INFO:root:left_df shape: (2496, 10)
INFO:root:right_df shape: (2563, 8)
INFO:root:joining on left_key: ['brinnummer_duo', 'opleidingsnaam_duo', 'soortopleiding_duo']
INFO:root:joining on right_key: ['brinnummer_sdb', 'naamopleiding_sdb', 'soortopleiding_sdb']
INFO:root:Join result
both          2232
right_only       0
left_only        0
Name: _merge, dtype: int64
INFO:root:joined df: (2232, 18)


In [20]:
import re
import string
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer


stopwords_nl = set(stopwords.words("dutch"))
punctuation = string.punctuation

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def clean_string_fields_en(x):

    x = (
        x.lower()
        .replace("\r\n\r\n", " ")
        .replace(" \xa0", "")
        .replace("?s", "")
        .replace("’", "")
        .replace("‘", "")
        .replace("“", "")
        .replace("”", "")
        .replace("—", "")
    )
    x = "".join(word for word in x if word not in punctuation)
    x = word_tokenize(x)
    x = " ".join(word for word in x if word not in stopwords_nl)
    x = re.sub(r"\d+", "", x)
    x = stemmer.stem(x)
    x = lemmatizer.lemmatize(x)

    return x

In [23]:
test['kortebeschrijvingopleiding_sdb'] = test['kortebeschrijvingopleiding_sdb'].fillna('geen beschrijving').apply(clean_string_fields_en)

In [24]:
test

,brinnummer_duo,instellingsnaam_duo,opleidingsnaam_duo,croho_onderdeel,croho_subonderdeel,soortopleiding_duo,man,vrouw,tot_2019_duo,gemeentenummer_duo,cluster_sdb,sector_sdb,kortebeschrijvingopleiding_sdb,eerstejaarsaantal_sdb,studentenaantal_sdb
0,00DV,Protestantse Theologische Universiteit,Gemeentepredikant,taal en cultuur,n.v.t. (taal en cultuur),master,13.0,2.0,15.0,"[14, 363]",Theologie en Levensbeschouwing,Kunst en Cultuur,predikant kerk slag zie jij jezelf pionieren g...,18.0,103.0
1,00DV,Protestantse Theologische Universiteit,Predikantschap,taal en cultuur,n.v.t. (taal en cultuur),master,5.0,2.0,7.0,"[14, 363]",Theologie en Levensbeschouwing,Kunst en Cultuur,predikant kerk slag zie jij jezelf pionieren s...,7.0,38.0
2,00DV,Protestantse Theologische Universiteit,Theologie (joint degree),taal en cultuur,n.v.t. (taal en cultuur),bachelor,6.0,5.0,11.0,"[14, 363]",Theologie en Levensbeschouwing,Kunst en Cultuur,gefascineerd bent geloof mensen doet mensen ge...,31.0,109.0
3,00DV,Protestantse Theologische Universiteit,Verdieping,taal en cultuur,n.v.t. (taal en cultuur),master,8.0,7.0,15.0,"[14, 363]",Theologie en Levensbeschouwing,Kunst en Cultuur,theologische kennis verdiepen theologie vooral...,11.0,NaN
4,00IC,Katholieke PABO Zwolle,Opleiding tot leraar Basisonderwijs,onderwijs,leraar basisonderwijs,bachelor,47.0,147.0,194.0,[193],Leraar basisonderwijs (PABO),Onderwijs en Opvoeding,kpz kleine hogeschool ca studenten waar direc...,250.0,698.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2227,31FR,NHL Stenden Hogeschool,Tourism Management,economie,n.v.t. (economie),associate degree,14.0,44.0,58.0,"[106, 114, 119, 80, 93, 14, 363, 193]",Toerisme en Vrijetijdskunde,Economie en Bedrijf,creatief communicatief graag werken toerisme r...,20.0,52.0
2228,31FR,NHL Stenden Hogeschool,Tourism Management,economie,n.v.t. (economie),bachelor,34.0,108.0,142.0,"[106, 114, 119, 80, 93, 14, 363, 193]",Toerisme en Vrijetijdskunde,Economie en Bedrijf,kies tourism management nhl stenden hogeschool...,148.0,668.0
2229,31FR,NHL Stenden Hogeschool,Vaktherapie,gedrag en maatschappij,n.v.t. (gedrag en maatschappij),bachelor,2.0,45.0,47.0,"[106, 114, 119, 80, 93, 14, 363, 193]",Creatieve Therapie,Gedrag en Maatschappij,kies vaktherapie nhl stenden hogeschool leer j...,71.0,212.0
2230,31FR,NHL Stenden Hogeschool,Werktuigbouwkunde,techniek,n.v.t. (techniek),bachelor,67.0,4.0,71.0,"[106, 114, 119, 80, 93, 14, 363, 193]",Werktuigbouwkunde,Techniek,kies werktuigbouwkunde nhl stenden hogeschool ...,28.0,90.0


In [39]:
test.opleidingsnaam_duo.value_counts().keys().tolist()

['Opleiding tot leraar Basisonderwijs',
 'Social Work',
 'Human Resource Management',
 'Accountancy',
 'Finance & Control',
 'Bedrijfskunde',
 'Technische Bedrijfskunde',
 'Opleiding tot Verpleegkundige',
 'International Business',
 'Werktuigbouwkunde',
 'Pedagogiek',
 'Filosofie',
 'Communicatie',
 'Ondernemerschap & Retail Management',
 'HBO-ICT',
 'Informatica',
 'Geneeskunde',
 'Bouwkunde',
 'Elektrotechniek',
 'HBO - Rechten',
 'Muziek',
 'Chemie',
 'Sociologie',
 'Rechtsgeleerdheid',
 'Theologie',
 'Civiele Techniek',
 'Geschiedenis',
 'Fiscaal Recht',
 'Biologie en Medisch Laboratoriumonderzoek',
 'Psychologie',
 'Management in de Zorg',
 'Facility Management',
 'Pedagogische Wetenschappen',
 'Docent Muziek',
 'Communication and Multimedia Design',
 'Pedagogisch Educatief Professional',
 'Opleiding tot Fysiotherapeut',
 'Tourism Management',
 'Educational Needs',
 'Logistics Management',
 'Autonome Beeldende Kunst',
 'Advanced Nursing Practice',
 'Docent Beeldende Kunst en Vormg

In [106]:
keywords = ['data', 'Intelligence', 'Data Science', 'Kunstmatige Intelligentie', 'computer science',
            'analytic', 'analist', 'artificial',
            'developer', 'online',  'design', 'media', 'multimedia', 'online', 'marketing', 'e-commerce',
            'grafisch', 'web', 'software', 'hardware', 'ontwikkelaar'
            'creative', 'digital', 'ontwerp',
            'information', 'ict', 'cyber', 'hack' ,'engineer', 'informatica', 'systeem', 'beheerder'
            'Technische', 'wiskunde', 'Mathematics', 'applied', 'quantitative',
            'computer', 'Technology']

In [107]:
cols_to_check = ['opleidingsnaam_duo', 'croho_onderdeel', 'cluster_sdb', 'kortebeschrijvingopleiding_sdb']
for c in cols_to_check:
    t = test[test[c].str.contains("|".join(keywords), flags=re.IGNORECASE)]

In [108]:
t

,brinnummer_duo,instellingsnaam_duo,opleidingsnaam_duo,croho_onderdeel,croho_subonderdeel,soortopleiding_duo,man,vrouw,tot_2019_duo,gemeentenummer_duo,cluster_sdb,sector_sdb,kortebeschrijvingopleiding_sdb,eerstejaarsaantal_sdb,studentenaantal_sdb
8,00MF,Hogeschool voor de Kunsten Utrecht,Docent Beeldende Kunst en Vormgeving,onderwijs,lerarenopleidingen op het gebied van de kunst,bachelor,4.0,26.0,30.0,"[307, 344]",Kunstacademie,Kunst en Cultuur,opleiding bachelor fine art and design educati...,53.0,178.0
20,01VU,Christelijke Hogeschool Windesheim,Bouwkunde,techniek,n.v.t. (techniek),bachelor,137.0,59.0,196.0,"[34, 80, 14, 153, 193, 344]",Bouwkunde,Techniek,mooie functionele gebouwen ontwerpen uitwerken...,199.0,489.0
21,01VU,Christelijke Hogeschool Windesheim,Civiele Techniek,techniek,n.v.t. (techniek),bachelor,51.0,8.0,59.0,"[34, 80, 14, 153, 193, 344]",Civiele techniek,Techniek,erasmusbrug oosterscheldekeerkring autowegen f...,84.0,266.0
22,01VU,Christelijke Hogeschool Windesheim,Communicatie,economie,n.v.t. (economie),bachelor,34.0,109.0,143.0,"[34, 80, 14, 153, 193, 344]",Communicatie,Taal en Communicatie,leren online offline media pr marketing teksts...,40.0,104.0
23,01VU,Christelijke Hogeschool Windesheim,Communicatie,economie,n.v.t. (economie),bachelor,34.0,109.0,143.0,"[34, 80, 14, 153, 193, 344]",Communicatie,Taal en Communicatie,opleiding leer bedrijfsdoelstellingen vertaalt...,109.0,380.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2196,31FR,NHL Stenden Hogeschool,Ocean Technology,techniek,n.v.t. (techniek),bachelor,14.0,6.0,20.0,"[106, 114, 119, 80, 93, 14, 363, 193]",Zee- en Luchtvaart,Techniek,kies opleiding ocean technology nhl stenden ho...,12.0,90.0
2199,31FR,NHL Stenden Hogeschool,Online Contentcreator,economie,n.v.t. (economie),associate degree,13.0,40.0,53.0,"[106, 114, 119, 80, 93, 14, 363, 193]",Mediastudies,Taal en Communicatie,jij mooie content maken allerlei online platfo...,12.0,12.0
2213,31FR,NHL Stenden Hogeschool,Opleiding tot leraar voortgezet onderwijs van ...,onderwijs,n.v.t. (onderwijs),bachelor,16.0,17.0,33.0,"[106, 114, 119, 80, 93, 14, 363, 193]",Lerarenopleidingen exact,Onderwijs en Opvoeding,kies leraar wiskunde nhl stenden hogeschool st...,43.0,201.0
2225,31FR,NHL Stenden Hogeschool,Technische Informatica,techniek,n.v.t. (techniek),bachelor,4.0,0.0,4.0,"[106, 114, 119, 80, 93, 14, 363, 193]",Informatica,Exact en Informatica,kies technische informatica nhl stenden hogesc...,16.0,54.0
